In [1]:
#pip install numpy===1.22.2 ipywidgets torch torchvision===0.11.3 pytorch-ignite
#torch currently not supported with python 3.10, downgrading to python 3.9.7 possibly required

In [2]:
#imports
import numpy as np
import torch
from torch import optim, nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, datasets, transforms
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

In [3]:
#"SETUP" To be removed
# Dataset  will be loaded further down.

# Set up hyperparameters.
lr = 0.001
batch_size = 128
num_epochs = 1

# Set up logging.
print_every = 1  # batches

# Set up device.
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
#dataset and preprocessing
#data ingestion
dataset = getattr(datasets, 'MNIST')

#data preparation
transform = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224), 
    transforms.ToTensor(), 
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # grayscale to RGB
])

#data seperation
# Wrap in data loader.
training_dataset = dataset("./data", train=True, download=True, transform=transform)
#training_dataset = datasets.MNIST("./data", train=True, download=True, transform=transform)
testing_dataset = dataset("./data", train=False, download=True, transform=transform)

if use_cuda:
    kwargs = {"pin_memory": True, "num_workers": 1}
else:
    kwargs = {}

train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=False, **kwargs)
val_loader = None

print(training_dataset.__len__)    
print(testing_dataset.__len__)

<bound method MNIST.__len__ of Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Lambda()
           )>
<bound method MNIST.__len__ of Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Lambda()
           )>


In [5]:
#model
# Set up model, loss, optimizer.
model = models.AlexNet()
num_classes = 1000
model = model.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


In [6]:
#training
# Set up pytorch-ignite trainer and evaluator.
trainer = create_supervised_trainer(
    model,
    optimizer,
    loss_func,
    device=device,
)
metrics = {
    "accuracy": Accuracy(),
    "loss": Loss(loss_func),
}
evaluator = create_supervised_evaluator(
    model, metrics=metrics, device=device
)

@trainer.on(Events.ITERATION_COMPLETED(every=print_every))
def log_batch(trainer):
    batch = (trainer.state.iteration - 1) % trainer.state.epoch_length + 1
    print(
        f"Epoch {trainer.state.epoch} / {num_epochs}, "
        f"batch {batch} / {trainer.state.epoch_length}: "
        f"loss: {trainer.state.output:.3f}"
    )

@trainer.on(Events.EPOCH_COMPLETED)
def log_epoch(trainer):
    print(f"Epoch {trainer.state.epoch} / {num_epochs} average results: ")

    def log_results(name, metrics, epoch):
        print(
            f"{name + ':':6} loss: {metrics['loss']:.3f}, "
            f"accuracy: {metrics['accuracy']:.3f}"
        )

    # Train data.
    evaluator.run(train_loader)
    log_results("train", evaluator.state.metrics, trainer.state.epoch)
    
    # Val data.
    if val_loader:
        evaluator.run(val_loader)
        log_results("val", evaluator.state.metrics, trainer.state.epoch)

    # Test data.
    if test_loader:
        evaluator.run(test_loader)
        log_results("test", evaluator.state.metrics, trainer.state.epoch)

    print()
    print("-" * 80)
    print()

# Start training.
trainer.run(train_loader, max_epochs=num_epochs)

Epoch 1 / 1, batch 1 / 469: loss: 6.909
Epoch 1 / 1, batch 2 / 469: loss: 4.979
Epoch 1 / 1, batch 3 / 469: loss: 55.579
Epoch 1 / 1, batch 4 / 469: loss: 5.618
Epoch 1 / 1, batch 5 / 469: loss: 4.048
Epoch 1 / 1, batch 6 / 469: loss: 4.190


Engine run is terminating due to exception: 


KeyboardInterrupt: 